## AERO3000 Space Systems Design

Faculty of Science and Engineering &nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp; Space Science and Technology Centre &nbsp;&nbsp;|&nbsp;&nbsp; CRICOS Provider Code 00301J

# MoonBrix Power Budget
Modelling the electrical generation, storage and consumption of our space system.

# Index of important variables
1. [Timing of different modes](#enter-the-timing-of-the-different-modes)
2. [Solar Availability](#solar-availability)
3. [Solar safety factor](#determine-cell-area)
4. [Power modelling](#power-modelling-of-nominal-operations)
5. [Cell chemistry](#chose-a-cell-chemistry)
6. [Battery Mass](#battery-mass)
7. [Total Mass](#total-mass)

### Import Modules

In [49]:
from astropy.time import Time #import astropy so we can use the astropy.Time class to set our epoch times
from astropy import units as u #import the astropy units package so we can use real world units conveniently
import numpy as np #import our maths library numpy under the alias np

from openpyxl import Workbook #import the openpyxl library so we can create excel spreadsheets

### Calculate Your Power Generation Over Time
This code assumes you are generating at the begining of the modelled time period.

Update this code to model the power generation for your mission.

<mark>Not sure if we need this as the operational period is the lunar day -> constantly in the sun</mark>

In [50]:
duration_in_min = 60*15
timestep_in_min = 5
t = np.arange(0, duration_in_min, timestep_in_min)*u.min
print(t)

generation_stop = 15*u.min
generation_restart = 110*u.min

illumination_status_list = [] #empty list

for i in range(len(t)):
    time = t[i]
    if ((generation_stop<=time) & (time<generation_restart)): #panels retracted or not tracking the Sun
        panels_illuminated = False
    else:
        panels_illuminated = True
    illumination_status_list.append(panels_illuminated)
    
in_sunlight_array = np.asarray(illumination_status_list)
print(in_sunlight_array)

[  0.   5.  10.  15.  20.  25.  30.  35.  40.  45.  50.  55.  60.  65.
  70.  75.  80.  85.  90.  95. 100. 105. 110. 115. 120. 125. 130. 135.
 140. 145. 150. 155. 160. 165. 170. 175. 180. 185. 190. 195. 200. 205.
 210. 215. 220. 225. 230. 235. 240. 245. 250. 255. 260. 265. 270. 275.
 280. 285. 290. 295. 300. 305. 310. 315. 320. 325. 330. 335. 340. 345.
 350. 355. 360. 365. 370. 375. 380. 385. 390. 395. 400. 405. 410. 415.
 420. 425. 430. 435. 440. 445. 450. 455. 460. 465. 470. 475. 480. 485.
 490. 495. 500. 505. 510. 515. 520. 525. 530. 535. 540. 545. 550. 555.
 560. 565. 570. 575. 580. 585. 590. 595. 600. 605. 610. 615. 620. 625.
 630. 635. 640. 645. 650. 655. 660. 665. 670. 675. 680. 685. 690. 695.
 700. 705. 710. 715. 720. 725. 730. 735. 740. 745. 750. 755. 760. 765.
 770. 775. 780. 785. 790. 795. 800. 805. 810. 815. 820. 825. 830. 835.
 840. 845. 850. 855. 860. 865. 870. 875. 880. 885. 890. 895.] min
[ True  True  True False False False False False False False False False
 False Fa

### Plot Illumination Status Over Time

In [51]:
# import plotly #import our graphing module plotly
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(x=t, y=in_sunlight_array)) #create a scatter plot from the in_eclipse_theta_array and in_eclipse_array
fig.layout.template = "plotly_dark" #activate the dark theme, comment out for light theme if you want

fig.update_yaxes(autorange="reversed") #flip y axis, comment out to unflip if required

fig.update_layout(title="Illumination Status Over Time (Simulated Period: {} min)".format(duration_in_min), #add figure title and axis titles
                  xaxis_title="t ({})".format(t.unit),
                  yaxis_title="In Sunlight",
                  )

fig.show() #display figure

### Get data from excel file

In [52]:
from openpyxl import load_workbook
wb = load_workbook(filename = 'Spacecraft Operational Modes.xlsx', data_only=True)

# grab the active worksheet
ws = wb.active

### Enter Power Draws for Different Operational Modes

In [53]:
p_idle = ws['D12'].value* u.W #idle power consumption of the spacecraft
print("Idle Power: ", p_idle)

p_collecting = ws['F12'].value*u.W #using the arm to collect regolith
print("Collecting Power: ", p_collecting)

p_sintering = ws['H12'].value*u.W #heating up the regolith to sinter
print("Sintering Power: ", p_sintering)

p_cooling = ws['J12'].value*u.W #cooling down heated regolith
print("Cooling Power: ", p_cooling)

p_testing = ws['L12'].value*u.W #compressive test on brick and imaging
print("Testing Power: ", p_testing)

Idle Power:  60.0 W
Sintering Power:  805.0 W
Cooling Power:  65.0 W
Testing Power:  235.0 W


### Enter the Timing of the Different Modes

In [54]:
power_consumption = np.ones(t.shape)*u.W #array of p_data_processing, same shape as t
collecting_start = 5*u.min
collecting_duration = 30*u.min

sintering_start = 60*u.min
sintering_duration = 5*u.hour

cooling_start = 360*u.min
cooling_duration = 3*u.hour

testing_start = 560*u.min
testing_duration = 1*u.hour

### Calculate Consumption

In [55]:
for i in range(len(t)):
    time = t[i]
    if ((collecting_start<=time) & (time<collecting_start+collecting_duration)): #Collecting
        power_consumption[i] = p_collecting
    elif ((sintering_start<=time) & (time<sintering_start+sintering_duration)): #Sintering
        power_consumption[i] = p_sintering
    elif ((cooling_start<=time) & (time<cooling_start+cooling_duration)): #Cooling
        power_consumption[i] = p_cooling
    elif ((testing_start<=time) & (time<testing_start+testing_duration)): #comms contact
        power_consumption[i] = p_testing
    else:
        power_consumption[i] = p_idle

### Plot the Power Consumption Over the Relevant Time Period

In [56]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=1)
fig.layout.template = 'plotly_dark' #activate the dark theme, comment out for light theme if you want

fig.add_trace(go.Scatter(x=t, y=power_consumption, name="Power Consumption ({})".format(power_consumption.unit), line=dict(color="#EF553B"), showlegend=False), row=1, col=1)

fig.update_yaxes(title_text="Power Consumption ({})".format(power_consumption.unit), rangemode="tozero", row=1, col=1)

fig.update_xaxes(range=[0, duration_in_min], row=1, col=1)
fig.update_xaxes(title_text="Time ({})".format(t.unit), range=[0, duration_in_min], row=1, col=1)

fig.update_layout(title_text="Power Consumption Over Time (Simulated Period: {} min)".format(duration_in_min),
                  height=800)

fig.show() #display figure

## Power Generation
(This is similar to section 10.6.3 in [chapter 10 "Electrical Power Systems" in Fortescue et al. (2011)](https://onlinelibrary-wiley-com.dbgw.lis.curtin.edu.au/doi/pdf/10.1002/9781119971009.ch10) on page 352.)

First: calculate your average power consumption:

In [57]:
p_avg = np.average(power_consumption)
print("Average power consumption: p_avg = {:.1f}".format(p_avg))

p_solar_targ = p_avg


Average power consumption: p_avg = 335.7 W


#### Solar Availability
To do the next step we will need to know the availability of the generation source (in sunlight fraction for solar power):

In [ ]:
lattitude_offset = 20 #max lattitude error of the spacecraft

lunar_day = 29.53*u.day #lunar day in earth days
time_angle = 360/lunar_day.to(u.hour).value #degrees per hour of the lunar day

hours_from_noon = 20 #time that mission will be offset from noon, in hours

time_of_day_offset = time_angle*hours_from_noon #time of day offset of the spacecraft, 0 degrees means sun is overhead (12:00 local time)

total_offset = np.sqrt(lattitude_offset**2 + time_of_day_offset**2) #rough approximation of the angle between the spacecraft and the sun, in degrees
#accuracy is higher at smaller angles


frac_gen_available = np.cos(np.deg2rad(total_offset)) #fraction of time the panels are generating power
#spacrecrafts mission duration fits well within a whole lunar day, so we pick best time to run payload

print("Availabity of generation : frac_gen_available = {:.0%}".format(frac_gen_available))

Availabity of generation : frac_gen_available = 92%


Then make an initial estimate of the required power generation by dividing by the availability of the generation source.

Keep in mind that this initial guess doesn't take into account different factors degrading the generation output such as temperature or radiation dose effects (damage) over time or inefficiencies in the system.

In [59]:
p_gen_initial_estimate = p_solar_targ/frac_gen_available
print("Initial estimate of power generation capacity : p_gen_initial_estimate = {:.0f}".format(p_gen_initial_estimate))

Initial estimate of power generation capacity : p_gen_initial_estimate = 363 W


### Cell Selection
Select an [Azure Space](http://www.azurspace.com/index.php/en/) or [Rocket Lab SolAero](https://www.rocketlabusa.com/space-systems/solar/space-solar-cellscics/) **space** solar cell and calculate the **active** cell area required to deliver this power generation capcity.

Assume J_S = 1371 W/m$^2$ and that the panels remain normal to the Sun vector at all times.

Update the comment to include the cell assembly type and the link to the datasheet.

Print the result in centimetres squared.

In [60]:
# Update Solar radiaiton flux if not around 1 AU from the Sun
J_S = 1371*u.W/u.m**2 #solar radiation flux from Fortescue et al. (2011) page 359
eff_cell = 30.2/100 #<cell assembly type>, source: <cell assembly datasheet>

initial_estimate_cell_area = p_gen_initial_estimate/J_S/eff_cell
print("Initial estimate cell active area: initial_estimate_cell_area = {:.0f}".format(initial_estimate_cell_area.to(u.cm*u.cm)))

Initial estimate cell active area: initial_estimate_cell_area = 8771 cm2


Calculate the number of cell assemblies from the cell area if given in the datasheet. If not provided, you could assume cell area is 30.18 cm$^2$.

In [61]:
area_per_cell = 8*4*u.cm**2 #<cell assembly type>

#calculate number of cells, rounding up using the np.ceil() funciton
inital_estimate_n_cells =  np.ceil((initial_estimate_cell_area/area_per_cell).to(u.dimensionless_unscaled))

print("Initial estimate number of cells: inital_estimate_n_cells = {:.0f}".format(inital_estimate_n_cells))

Initial estimate number of cells: inital_estimate_n_cells = 275


### Calculate Losses

Assume battery charging is 90% efficient.

Assume degradation due to temperature rise is approximately 0.2 % per °C above room temperature (assumption is roughly based on [Rocket Lab SolAero ZTJ datasheet](https://www.rocketlabusa.com/assets/Uploads/RL-SolAero-Data-Sheet-ZTJ.pdf)). If you haven't done a thermal balance yet assume your cells are around 70 °C in an Earth orbit (although you should do a thermal balance calculation ASAP).

Assume a degradation due to radiation based on [Table 6.7 from Brown (2002) p. 339](https://ebookcentral.proquest.com/lib/curtin/reader.action?docID=3111499&ppg=352) and your mission parameters.

In [62]:
eff_charging = 90/100 #90% efficient solar tracking and cell charging
radiation_degradation_loss = 4/100 #<insert mission information>, based on Brown (2002) p. 339
temperature_loss = 17/100 #assumption rougly based on figure 6.18 on page 337 of Brown (2002)

### Determine Cell Area

Now pick your actual cell area or number of cells which should be informed by your initial estimates. (It should be larger, however.)

If you want, you can just pick you initial estimate for now, and then come back and refine it after the initial run of the model. An iterative approach like this is reasonable.

In [63]:
safety_factor_solar = 2

n_cells = np.round(inital_estimate_n_cells*safety_factor_solar)  #cells for space craft
cell_area = n_cells * area_per_cell
print(cell_area.to(u.m**2))

1.76 m2


### Calculate the Power Generation Over Time

Fill in the missing power generation capacity expression accounting for the losses below and then run the code.

Note that the expression for efficiencies will be different than the expression for losses.

In [64]:
# calculate your power generation capacity with losses
p_gen_capacity = (J_S*eff_cell*cell_area*(1-radiation_degradation_loss)*(1-temperature_loss)*eff_charging).to(u.W)
print("Power generation with losses: p_gen_capacity = {:.1f}".format(p_gen_capacity))

Power generation with losses: p_gen_capacity = 522.6 W


Now calculate your power generation over time array by multiplying the capacity (scalar) times the availability (array).

In [65]:
power_generation = p_gen_capacity*frac_gen_available
print("Power generation: power_generation = {:.1f}".format(power_generation.to(u.W)))

power_generation_array = power_generation * np.ones(t.shape)

Power generation: power_generation = 483.0 W


## Power Modelling of Nominal Operations
For now we are going to assume the batteries start empty and you have a very large battery capacity.

You should come back and improve these assumptions as you progress the design of your space system's electrical power system.

Fill in the missing expression to calculate the next energy storage level (it's a numerical integration of power over time) and then run the code below.

Also fill in the missing part of the expression to limit the maximum level of stored energy to the battery pack capacity.

In [66]:

energy_storage_capacity = 2000*u.W*u.h #energy storage limit (battery pack capacity)
e_0 = energy_storage_capacity #0*u.W*u.h # initial energy storage level

energy_stored = np.zeros(t.shape)*u.W*u.h #create array of zeroes the same shape as t

delta_t = t[1]-t[0]

for i in range(len(t)):
    time = t[i]
    if i==0:
        energy_stored[i] = e_0
    else:
        next_energy_storage_level = energy_stored[i-1]+(power_generation_array[i]-power_consumption[i])*delta_t #calculate the next energy storage level
        energy_stored[i] = np.clip(next_energy_storage_level, 0, energy_storage_capacity) #clamp energy storage

        
        #if(next_energy_storage_level > energy_storage_capacity):
        #    energy_stored[i] = energy_storage_capacity
        #else:
        #    energy_stored[i] = next_energy_storage_level

from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols=1)

print("Energy stored after one orbit: {:.2f} WH".format(energy_stored[-1].to(u.W*u.h).value))
print("Minimum storage state: {:.2f} WH".format(energy_stored.min().to(u.W*u.h).value))

if energy_stored.min() == 0:
    print("WARNING: energy production/storage too small \n")

print("Maximum storage state: {:.2f} WH".format(energy_stored.max().to(u.W*u.h).value))


fig.layout.template = 'plotly_dark' #activate the dark theme, comment out for light theme if you want

fig.add_trace(go.Scatter(x=t, y=power_generation_array, name="Power Generation (W)", line=dict(color="#636EFA"), showlegend=False), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=power_consumption, name="Power Consumption (W)", line=dict(color="#EF553B"), showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=energy_stored, name="Energy Stored (WH)", line=dict(color="#00CC96"), showlegend=False), row=3, col=1)

fig.update_yaxes(title_text="Power Generation (W)", rangemode="tozero", row=1, col=1)
fig.update_yaxes(title_text="Power Consumption (W)", rangemode="tozero", row=2, col=1)
fig.update_yaxes(title_text="Energy Stored (WH)", rangemode="tozero", row=3, col=1)

fig.update_xaxes(range=[0, duration_in_min], row=1, col=1)
fig.update_xaxes(range=[0, duration_in_min], row=2, col=1)
fig.update_xaxes(title_text="Time ({})".format(t.unit), range=[0, duration_in_min], row=3, col=1)

fig.update_layout(title_text="Power Generation, Power Consumption and Energy Storage Over Time ({:.2f} min)".format(duration_in_min),
                  height=1000,
                  showlegend=True,
                  legend=dict(traceorder="normal"),
                  barmode="stack")

fig.show() #display figure

fig.write_image("Mission_power_profile.png") 

Energy stored after one orbit: 2000.00 WH
Minimum storage state: 390.16 WH
Maximum storage state: 2000.00 WH


### Go Back and Select a Realistic Battery Pack Capacity and Initial Charge Level
Also check that you don't run out of power at any point. You may wish to move loads around (in time) if you have high power consumption when generation is not available.

Remember, you never what your space system to run out of power.

### Depth of Discharge
From the green trace above determine the depth of discharge and record it in the text box below as a percentage.

In [67]:
depth_d = energy_stored.min()/energy_stored.max()
depth_d = 1-depth_d
print("Depth of discharge: {:.1f}%".format(depth_d*100))

Depth of discharge: 80.5%


### Chose a Cell Chemistry
Write the chosen chemistry in the text box below.

### ~Projected Lifetime with Chosen Chemistry and Depth of Discharge~
~Update code for your chemistry and mission.~

This step doesn't make sense for missions without repetitive charge and discharge cycles (like a flyby or lander), so it probably doesn't apply.

Uncomment (Highlight, then press Ctrl + / ) and update the code below if it does apply to your mission.

In [68]:
# # Number of charge-discharge cycles per orbit
# charge_cycles_per_orbit = 1
# #The cell chemsitry cycle life at the chosen depth of discharge
# cell_cycle_life_at_dod = 2000
# battery_pack_lifetime = (charge_cycles_per_orbit*cell_cycle_life_at_dod*tau).to(u.yr)
# print("Battery Pack Lifetime: {:.2}".format(battery_pack_lifetime))

# end_of_life_capacity_ratio = 0.6
# eol_remaining_capacity = (energy_storage_capacity*end_of_life_capacity_ratio).to(u.W*u.hr)
# print("Battery Pack Capacity Remaining at End of Life: {:.0f} WH".format(eol_remaining_capacity.value))

### Add in a Safety Factor and Print your Panel and Battery Pack Capacity

In [69]:
#add your code here

### Battery Mass
Look up the energy density of your chosen chemistry, and calculate the approximate mass of your battery pack below. Feel free to rely on easily available sources for this, you don't necessarily need to use a high quality source for this step.

If you would like to look at some space battery cells, you can if you wish.

In [70]:
LiFePO_4_Specific_energy = 150 *u.W*u.h/u.kg #(90–160 Wh/kg)
LiFePO_4_Energy_density = 325 *u.W*u.h/u.L

battery_mass = energy_storage_capacity / LiFePO_4_Specific_energy #mass of the battery pack

print("Battery Pack Mass: {:.1f} kg".format(battery_mass.to(u.kg).value))

battery_volume = energy_storage_capacity / LiFePO_4_Energy_density #volume of the battery pack
print("Battery Pack Volume: {:.1f} L".format(battery_volume.to(u.L).value))

Battery Pack Mass: 13.3 kg
Battery Pack Volume: 6.2 L


## Total mass
Estimate the mass of your solar arrays. Account for the cells, the structure, and any interface layers or wiring if required.

Update your system's mass budget.

In [71]:
solar_cell_mass = 86 *u.mg/u.cm**2 #mass of the solar panel per unit area
solar_panel_mass = solar_cell_mass * cell_area #mass of the solar panel
print("Solar Panel Mass: {:.1f} kg".format(solar_panel_mass.to(u.kg).value))

print("Battery Pack mass: {:.1f} kg".format(battery_mass.to(u.kg).value))

peak_power = power_consumption.max() #maximum power consumption capacity
print("Peak Power: {:.1f} W".format(peak_power.to(u.W).value))

power_electronics_mass = 1 * u.kg #mass of the power electronics

power_mass = (battery_mass + solar_panel_mass + power_electronics_mass) #mass of the power system per watt of power generation
print("Power System Mass: {:.1f} kg\n".format(power_mass.to(u.kg).value))

Solar Panel Mass: 1.5 kg
Battery Pack mass: 13.3 kg
Peak Power: 805.0 W
Power System Mass: 15.8 kg



### Saving to excel file

In [72]:
wb_saving = load_workbook(filename = 'Spacecraft subsystems.xlsx') #create a new workbook
ws_saving = wb_saving.active #create a new worksheet in the workbook

ws_saving['A1'] = "Subsystem" #set the column titles
ws_saving['B1'].value = "Mass (Kg)" #set the column titles
ws_saving['C1'].value = "Volume (L)" #set the column titles
ws_saving['D1'].value = "Notes" #set the mass of the solar panels

ws_saving['A2'].value = "Power subsystem"
ws_saving['B2'].value = power_mass.value #set the mass of the power subsystem
ws_saving['C2'].value = battery_volume.value #set the volume of the power subsystem
ws_saving['D2'].value = "Solar panel volume not included" #set the mass of the solar panels

wb_saving.save('Spacecraft subsystems.xlsx') #save the workbook to a file
wb_saving.close() #close the workbook

# References

[Brown, Charles D. 2002. *Elements of Spacecraft Design*. Reston, Virginia, USA: American Institute of Aeronautics and Astronautics.](https://ebookcentral.proquest.com/lib/curtin/detail.action?pq-origsite=primo&docID=3111499)

[Fortescue, Peter, Graham Swinerd, and John Stark, eds. 2011. *Spacecraft Systems Engineering*. Chichester, UK: John Wiley & Sons.](https://onlinelibrary.wiley.com/doi/pdf/10.1002/9781119971009)